## 05. 콘텐츠 기반 필터링 실습 - TMDB 5000 영화 데이터 세트
TMDB 5000 영화 데이터 세트 [다운로드](https://www.kaggle.com/tmdb/tmdb-movie-metadata)

### 장르 속성을 이용한 영화 콘텐츠 기반 필터링
- 사용자가 긍정적인 평가를 내린 영화(상품/서비스)이 있다면 그와 비슷한 특성, 구성 요소 등을 가진 다른 영화(상품/서비스)을 추천하는 것
- 상품/서비스를 구성하는 다양한 콘텐츠를 기반으로 상품/서비스 간 유사성을 판단함

In [2]:
import numpy as np
import pandas as pd
import warnings 

warnings.filterwarnings('ignore')

In [4]:
import os
os.listdir(PATH)

['tmdb_5000_credits.csv', 'tmdb_5000_movies.csv']

In [15]:
PATH = './tmdb_movie/'

movies = pd.read_csv(PATH + 'tmdb_5000_movies.csv')
print(movies.shape)
print(movies.columns)
movies.head(1)

(4803, 20)
Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""sp...",en,Avatar,"In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, ...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289}, {""name"": ""Twentieth Century Fox Film Corporatio...","[{""iso_3166_1"": ""US"", ""name"": ""United States of America""}, {""iso_3166_1"": ""GB"", ""name"": ""United ...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,Enter the World of Pandora.,Avatar,7.2,11800


콘텐츠 기반 필털이 추천 분석에서 사용할 주요 컬럼만 추출
- title : 영화 제목
- genres : 영화가 속한 여러 장르
- vote_average : 평균 평점
- vote_count : 평점 투표 수
- popularity : 영화 인기
- keywords : 영화를 설명하는 주요 키워드 문구
- overview : 영화에 대한 개요 설명

In [16]:
movies_df = movies[['id','title', 'genres', 'vote_average', 'vote_count',
                 'popularity', 'keywords', 'overview']]

In [17]:
pd.set_option('max_colwidth',100)
movies_df[['genres','keywords']][:1]

,genres,keywords
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""sp..."


#### literal_eval()
칼럼 문자열을 분해해서 개별 장르의 파이썬 리스트 객체로 추출  
literal_eval() 처리를 해준 칼럼은 문자열이 아니라 실제 리스트 내부에 여러 장르 딕셔너리로 구성된 객체를 가짐

In [18]:
from ast import literal_eval

movies_df['genres'] = movies_df['genres'].apply(literal_eval)
movies_df['keywords'] = movies_df['keywords'].apply(literal_eval)

pd.set_option('max_colwidth',100)
movies_df[['genres','keywords']][:1]

,genres,keywords
0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {...","[{'id': 1463, 'name': 'culture clash'}, {'id': 2964, 'name': 'future'}, {'id': 3386, 'name': 'sp..."


apply() 함수로 장르명만 추출

In [19]:
movies_df['genres'] = movies_df['genres'].apply(lambda x : [ y['name'] for y in x ])
movies_df['keywords'] = movies_df['keywords'].apply(lambda x : [ y['name'] for y in x])

movies_df[['genres','keywords']].head(3)

,genres,keywords
0,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colony, society, space travel, futuristic, romance, spa..."
1,"[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india trading company, love of one's life, traitor, ship..."
2,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi6, british secret service, united kingdom]"


### 장르 콘텐츠 유사도 측정
genres를 문자열로 변경하여 이를 CountVectorizer로 피처 벡터화한 행렬 데이터 값을 코사인 유사도로 비교

1. 문자열로 반환된 genres 컬럼을 Count 기반으로 피처 벡터화 변환
2. genres 문자열을 피처 벡터화